## Managery kontekstu
Czasami alokujemy zasoby, które wymagają zwolnienia lub używamy obiektów, wymagających inicjalizacji i deinicjalizacji. Aby zapewnić, że deinicjalizacja lub zwolnienie zawsze nastapi możemy napisać taki kod:
```
try:
    obj = initialize()
    do_something(obj)
finally:
    deinitialize(obj)
```

Jest on jednak mniej czytelny niż kod:
```
with initialize() as obj:
    do_something(obj)
```
Gdzie zatem odbywa się deinicjalizacja? Cała tajemnica kryje się w konstrukcji `with ... as:`. W naszym przykładzie `initialize()` zwraca tzw. *manager kontekstu*, który zajmuje się zapewnieniem, by odpowiednia inicjalizacja i deinicjalizacja odbyły się odpowiednio na początku i na końcu bloku kodu, zawartego w konstrukcji `with...`. Czym jest ów `manager kontekstu`? Klasą implementującą następujący interfejs:
```
class FileOpener:
    def __init__(self, name):
        self.file_obj = open(file_name, method)
    
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        self.file_obj.close()
```
Metoda `__exit__` zostanie uruchomiona nawet, gdy w bloku `with...` zostanie rzucony wyjątek. Managery konttekstu mogą przydać się do:
- otwierania i zamykania plików
- wykonywania operacji alokujących pamięć poza interpreterem pythona, np. w bilbliotekach zaimplementowanych w C
- zakładania i zwalniania blokad
- startowania i stopowania różnych procesów w tle
- ...

Przykładem zastosowania managera contextu jest też użycie go do sprawdzania czy funkcja rzuca wyjątek w dobrym momencie - np.:

In [ ]:
def test_list_dir_returns_nothing_for_directory_with_no_files(tmp_dir_without_files):
    list_dir = make_traverser(tmp_dir_without_files)
    with pytest.raises(StopIteration):
        next(list_dir())

Aby uprościć pisanie managerów contekstu możemy też skorzystać z dekoratora `@contextlib.contextmanager`:

In [ ]:
from contextlib import contextmanager
from time import sleep

@contextmanager
def logger():
    print("Starting...")
    try:
        yield
    finally:
        print("... and done")

with logger():
    print("doing something")
    sleep(3)

### *Zadanie*
W pliku `context_managers.py` znajduje się kod, w którym należy uzupełnić brakujące managery kontekstu: `measure_transaction_perf` i `DatabaseTransaction`. Pierwszy z nich powinien mierzyć czas wykonania transakci bazodanowej, a drugi powinien umożliwiać jej bezpieczne rozpoczęcie i zakończenie. Jeśli podczas wykonywania operacji w transakcji zaszedł wyjątek, transakcja powinna zostać wycofana, w przeciwnym przypadku powinna zostać scommittowana do bazy. W `peewee` transakcje rozpoczyna się metodą `db.begin()`, wycofuje `db.rollback()` a committuje `db.commit()`.